In [ ]:
sentences = open('input.txt', encoding='utf-8').readlines()
tokenize_sentences = [sentence.split(' ') for sentence in sentences]

In [ ]:
pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.3/591.3 KB 48.8 MB/s eta 0:00:00


In [ ]:
from underthesea import word_tokenize

In [ ]:
with open('underthesea.txt', 'w', encoding='utf-8') as f:
    underthesea_sentences = []
    for sentence in sentences:
        word_list = word_tokenize(sentence, format="text")
        underthesea_sentences.append(''.join(word_list))
        for word in word_list: f.write(word + '\n')
        if sentence != sentences[-1]: f.write('\n')
    f.write('\n')
underthesea_sentences[0:3]

['Đồ_ăn hải_sản tươi ngon , các món canh nấu rất tuyệt .',
 'Mình hay ghé nhà_hàng Ngon ở Trần_Hưng_Đạo để dùng bữa với bạn_bè và đối_tác .',
 'Đồ ăn_tươi ngon chế_biến vừa_vặn , giá_cả hợp_lý cạnh quán Nghĩa_Ghẹ .']

In [ ]:
count_underthesea_compounds = 0
for sentence in underthesea_sentences:
    for word in sentence.split():
        if '_' in word: count_underthesea_compounds += 1
print('Số lượng từ ghép khi tách từ bằng thư viện underthesea:', count_underthesea_compounds)

Số lượng từ ghép khi tách từ bằng thư viện underthesea: 213


In [ ]:
with open('vietnamese.txt', 'r', encoding='utf-8') as f:
    manual_tokenize_sentences = []
    sentence = ''
    for word in f:
        if word == '\n': 
            manual_tokenize_sentences.append(sentence.strip())
            sentence = ''
        else: sentence += word.replace('\n', ' ')
manual_tokenize_sentences[0:51]

['Đồ_ăn hải_sản tươi ngon , các món canh nấu rất tuyệt .',
 'Mình hay ghé nhà_hàng Ngon ở Trần_Hưng_Đạo để dùng bữa với bạn_bè và đối_tác .',
 'Đồ_ăn tươi ngon chế_biến vừa_vặn , giá_cả hợp_lý cạnh quán Nghĩa_Ghẹ .',
 'Đồ_ăn ngon , hải_sản tươi , quán sạch , nhân_viên  dễ_thương  .',
 'Lần đầu_tiên ghé ăn nhưng rất hài_lòng , chắc_chắn là sẽ quay lại .',
 'Quán bán đủ món cháo lươn miến lươn , bún phở đầy_đủ , có nhiều lựa_chọn .',
 'Sân_vận_động đã được đầu_tư cải_tạo lại , sân_cỏ đẹp , khán_đài được làm lại đẹp hơn trước rất nhiều .',
 'Quảng_trường xanh , rộng và thoáng mát của thành_phố biển Quy_Nhơn , rất thích_hợp cho việc đi bộ .',
 'Một tổ_hợp khoa_học , nơi tổ_chức hội_thảo khoa_học thu_hút nhân_tài thế_giới .',
 'Khu_vui_chơi dành cho mọi_người có nhiều dụng_cụ tập_thể_dục , cây hoa được chăm_sóc tốt .',
 'Thiên_nhiên hoang_sơ , chưa được khai_phá nhiều nên vẫn còn nhiều nét mộc_mạc .',
 'Một địa_điểm hay cho các học_sinh thăm quan để hiểu sâu hơn giữa lý_thuyết và thực_tế cá

In [ ]:
count_manual_tokenize_compounds = 0
for sentence in manual_tokenize_sentences:
    for word in sentence.split():
        if '_' in word: count_manual_tokenize_compounds += 1
print('Số lượng từ ghép khi tách từ thủ công:', count_manual_tokenize_compounds)

Số lượng từ ghép khi tách từ thủ công: 206


In [ ]:
def count_correct_words(pred, source, n_grams=3):
    pred_words = pred.split()
    source_words = source.split()
    
    total_true, tp = 0, 0
    total_errors, fp = 0, 0
    
    idx = 0
    while idx < len(pred_words):
        if pred_words[idx] not in source_words[idx:(idx + n_grams)]: 
            if '_' in pred_words[idx]: fp += 1
            del pred_words[idx]
            total_errors += 1
        else: idx += 1
    
    idx = 0
    while idx < len(source_words):
        if source_words[idx] not in pred_words[idx:(idx + n_grams)]: 
            del source_words[idx]
        else: idx += 1
    
    if len(pred_words) < len(source_words): words = pred_words
    else: words = source_words
    
    for idx in range (len(words)):
        if pred_words[idx] == source_words[idx]:
            if '_' in pred_words[idx]: tp += 1 
            total_true += 1
                    
    return total_true, total_errors, tp, fp

In [ ]:
def tokenize_evaluation(pred, source, n_grams=3):
    total_true = 0
    total_errors = 0
    total_words = 0
    
    pred_tp = 0
    pred_fp = 0
    
    for pred_sentence, source_sentence in zip(pred, source):
        total_words += len(source_sentence.split())
        if pred_sentence != source_sentence:
            true, error, tp, fp = count_correct_words(pred_sentence, source_sentence, n_grams)
            total_true += true 
            total_errors += error
            pred_tp += tp
            pred_fp += fp
        else:
            for word in source_sentence.split():
                if '_' in word: pred_tp += 1
                total_true += 1
    return {
        'Accuracy': total_true / total_words, 
        'Precision': pred_tp / (pred_tp + pred_fp),
        'Recall': pred_tp / count_manual_tokenize_compounds,
        'True Positive': pred_tp, 
        'False Positive': pred_fp,
        'Total True': total_true, 
        'Total Errors': total_errors
    }

In [ ]:
from collections import defaultdict
import unicodedata as ud
import pandas as pd
import numpy as np
import math
import ast
import re

In [ ]:
underthesea_evaluation = tokenize_evaluation(underthesea_sentences, manual_tokenize_sentences)
pd.DataFrame(
    [underthesea_evaluation], 
    index = ['Underthesea']
).astype(object).T

,Underthesea
Accuracy,0.899721
Precision,0.885714
Recall,0.902913
True Positive,186
False Positive,24
Total True,646
Total Errors,69
